In [55]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag

In [41]:
def clean(string):
    cleaned = string.replace("<s>", "").replace("<P>", "")
    cleaned = re.sub("\s+([.,?!'])", "\\1", cleaned)
    return cleaned.strip()

In [72]:
def find_nested_relation(
    relation_type, relation, cur_depth=1, max_depth=3
):
    if relation is None or cur_depth > max_depth:
        return None, None
    elif relation_type == relation.type:
        return relation, cur_depth
    else:
        left_result, depth = find_nested_relation(
            relation_type, relation.left_child, cur_depth + 1
        )
        if left_result is None:
            return find_nested_relation(
                relation_type, relation.right_child, cur_depth + 1
            )
        else:
            return left_result, depth

In [15]:
# Testing

# %run ./relation_extraction.ipynb

# relation = Relation(
#     "Explanation", 
#     1, 
#     None,
#     Relation(
#         "Background",
#         2,
#         None,
#         None,
#         None
#     ),
#     None
# )

# print(find_nested_relation("Background", None))

# print(find_nested_relation("Explanation", relation))

# print(find_nested_relation("Background", relation))

# relation = Relation(
#     "Explanation", 
#     1, 
#     None,
#     Relation(
#         "Background",
#         2,
#         None,
#         None,
#         None
#     ),
#     Relation(
#         "Background",
#         3,
#         None,
#         Relation(
#             "Elaboration",
#             4,
#             None,
#             None,
#             None
#         ),
#         None
#     )
# )

# print(find_nested_relation("Background", relation))

# print(find_nested_relation("Elaboration", relation))

None
Relation(type='Explanation', left=1, right=None, left_child=Relation(type='Background', left=2, right=None, left_child=None, right_child=None), right_child=None)
Relation(type='Background', left=2, right=None, left_child=None, right_child=None)
Relation(type='Background', left=2, right=None, left_child=None, right_child=None)
Relation(type='Elaboration', left=4, right=None, left_child=None, right_child=None)


In [28]:
with open("connectives.txt", "rt") as f:
    connectives = set(
        [line.strip() for line in f.readlines()]
    )

In [43]:
punctuation = set([".", ",", "!", "?", ";"])

In [44]:
def remove_trailing_punctuation(string):
    if len(string) == 0 or string[-1] not in punctuation:
        return string
    else:
        return string[:-1]

In [64]:
def remove_leading_punctuation(string):
    if len(string) == 0 or string[0] not in punctuation:
        return string
    else:
        return string[1:].strip()

In [1]:
def lowercase_first_letter(string):
    return string[:1].lower() + string[1:]


def uppercase_first_letter(string):
    return string[:1].upper() + string[1:]

In [66]:
def trim_connective(string):
    lower = string.lower()
    for c in connectives:
        if lower[:len(c)] == c:
            if (
                c != "last" 
                or (len(lower) > len(c) and lower[len(c)] == ",")
            ):
                return remove_leading_punctuation(
                    string[len(c):]
                )
            else:
                return string
    return string